In [3]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm

In [4]:
import data_processing as dp

In [5]:
# importing the module
import json
  
# Opening JSON file
f = open('json_datasets/train.json', 'r')

raw_train = json.load(f)

vocab = dp.Vocabulary()
vocab.parseText(raw_train)

dataset = dp.text2GraphDataset(raw_json_file = raw_train)
dataloader = dp.getBatches(vocab, dataset, batch_size = 8, shuffle = False)

Creating empty vocabulary object
Finished Parsing Text
Creating custom dataset for T2G task
Creating empty vocabulary object
Finished Parsing Text
Finished processing raw json file


In [6]:
for (x,y) in zip(dataloader[0]['text'],dataloader[0]['entity_inds']):
	print([x[z[0]:z[1]] for z in y])

[tensor([1493, 1497]), tensor([1492, 1493, 1494, 1495, 1496])]
[tensor([1493, 1497]), tensor([1492, 1493, 1494, 1495, 1496])]
[tensor([1493, 1497]), tensor([1493])]
[tensor([1493, 1497]), tensor([1498])]
[tensor([1493, 1497]), tensor([1498])]
[tensor([1493, 1497]), tensor([1498])]
[tensor([1493, 1497]), tensor([1499])]
[tensor([1493, 1497]), tensor([1499])]


In [7]:
inp_types = len(vocab.entities.wordlist) + len(vocab.text.wordlist)
rel_types = len(vocab.relations.wordlist)

In [8]:
dataloader[0]

{'entity_inds': [[(2, 4), (5, 10)],
  [(1, 3), (7, 12)],
  [(1, 3), (7, 8)],
  [(1, 3), (4, 5)],
  [(1, 3), (8, 9)],
  [(1, 3), (4, 5)],
  [(1, 3), (6, 7)],
  [(4, 6), (7, 8)]],
 'text_lengths': [12, 14, 10, 11, 15, 12, 9, 10],
 'entity_lengths': [2, 2, 2, 2, 2, 2, 2, 2],
 'text': tensor([[   2,    4, 1493, 1497,    6, 1492, 1493, 1494, 1495, 1496,    8,    3,
             0,    0,    0],
         [   2, 1493, 1497,    9,    4,   10,    6, 1492, 1493, 1494, 1495, 1496,
             8,    3,    0],
         [   2, 1493, 1497,    9,    4,   10,    6, 1493,    8,    3,    0,    0,
             0,    0,    0],
         [   2, 1493, 1497,   11, 1498,   12,   13,   14,   15,    8,    3,    0,
             0,    0,    0],
         [   2, 1493, 1497,   11,   16,   17,   18,    6, 1498,   12,   13,   19,
            15,    8,    3],
         [   2, 1493, 1497,   11, 1498,   12,   13,    4,   14,   15,    8,    3,
             0,    0,    0],
         [   2, 1493, 1497,   11,   20,   21, 1499,  

In [1]:
model = ModelLSTM(inp_types, rel_types, 100)

model.forward(dataloader[0])

NameError: name 'ModelLSTM' is not defined

In [100]:
class ModelLSTM(nn.Module):
	def __init__(self, input_types, relation_types, model_dim, dropout = 0.5):
		super().__init__()

		self.word_types = input_types
		self.relation_types = relation_types
		self.dropout = dropout
		self.model_dim = model_dim

		self.emb = nn.Embedding(input_types, self.model_dim) # 40000 because we use the Bert tokenizer
		self.lstm = nn.LSTM(self.model_dim, self.model_dim//2, batch_first=True, bidirectional=True, num_layers=2)
		self.relation_layer1 = nn.Linear(self.model_dim , self.model_dim)
		self.relation_layer2 = nn.Linear(self.model_dim , self.model_dim)
		self.drop = nn.Dropout(self.dropout)
		self.projection = nn.Linear(self.model_dim , self.model_dim)
		self.decoder = nn.Linear(self.model_dim , self.relation_types)
		self.layer_norm = nn.LayerNorm(self.model_dim)

		self.init_params()

	def init_params(self):
		nn.init.xavier_normal_(self.relation_layer1.weight.data)
		nn.init.xavier_normal_(self.relation_layer2.weight.data)
		nn.init.xavier_normal_(self.projection.weight.data)
		nn.init.xavier_normal_(self.decoder.weight.data)

		nn.init.constant_(self.relation_layer1.bias.data , 0)
		nn.init.constant_(self.relation_layer2.bias.data , 0)
		nn.init.constant_(self.projection.bias.data , 0)
		nn.init.constant_(self.decoder.bias.data , 0)

	def forward(self, batch):
		sents = batch['text']
		sents, (c_0, h_0) = self.lstm(self.emb(sents))

		bs, _, hidden_dim = sents.shape
		max_ents = max([len(x) for x in batch['entity_inds']])
		
		cont_word_mask = sents.new_zeros(bs, max_ents)
		cont_word_embs = sents.new_zeros(bs, max_ents, hidden_dim)

		for b, (sent,entind) in enumerate(zip(sents,batch['entity_inds'])):
			for n_ent, wordemb in enumerate([sent[z[0]:z[1]] for z in entind]):
				cont_word_embs[b, n_ent] = torch.mean(wordemb, dim = 0)
				cont_word_mask[b, n_ent] = 1

		# bs x max_ents x model_dim
		cont_word_embs = self.layer_norm(cont_word_embs)

		rel1 = self.relation_layer1(cont_word_embs)
		rel2 = self.relation_layer2(cont_word_embs)

		#bs x max_ents x max_ents x model_dim
		out = rel1.unsqueeze(1) + rel2.unsqueeze(2)

		out = F.relu(self.drop(out))
		out = F.relu(self.projection(out))
		out = self.decoder(out)

		out = out * cont_word_mask.view(bs,max_ents,1,1) * cont_word_mask.view(bs,1,max_ents,1)

		return torch.log_softmax(out, -1)